june 29 cntk session azure ml meetup - insprired by dr james mccaffrey article in jan 2017 msdn on CNTK
this is a python version of the small problem originally done in brainscript
any bugs or issues here are the author's - not dr mccaffrey's
jimw

In [7]:
from __future__ import print_function

import cntk as C
 
from   cntk.learners import sgd, learning_rate_schedule, UnitType
from   cntk.logging import ProgressPrinter        # kind of a logger
from   cntk.layers import Dense, Sequential       #preworked layers we can choose to use (or ignore)
from   cntk.io import CTFDeserializer, MinibatchSource,StreamDef,StreamDefs
from   cntk.io import INFINITELY_REPEAT 
from   cntk.train.training_session import *

import numpy as np

input_dim    = 2     #2 features to input per sample
output_dim   = 3     #3 classes/labels each sample
layers_dim   = 1     #1 hidden dimension as this is just for starting
hidden2_dim  = 5     #5 hidden nodes in the 1 hidden dimension

input_var    = C.input(input_dim,np.float32)
label_var    = C.input(output_dim,np.float32)
inputfile    = "C:\\Users\\jiwillia\\Documents\\meetup\\deepL\\CNTK-session2\\TrainData.txt"



sample of a section of the training file (traindata.txt) can also be used as-is for BrainScript
|features 3.0 1.0 |labels 1 0 0
|features 5.0 2.0 |labels 1 0 0
|features 6.0 7.0 |labels 1 0 0
|features 7.0 4.0 |labels 1 0 0
|features 3.0 5.0 |labels 0 1 0
|features 4.0 4.0 |labels 0 1 0




In [8]:
#reader function - returns a minibatch reader to use in training and testing of a network and learner
def create_reader(pathtofile,is_training,inputsdim,outputsdim):
    return MinibatchSource(CTFDeserializer(pathtofile,StreamDefs(labels=StreamDef(field='labels',shape=outputsdim,
            is_sparse=False),
            features=StreamDef(field='features',shape=inputsdim,is_sparse=False))),randomize=is_training,
            max_sweeps=INFINITELY_REPEAT if is_training else 1)


In [9]:
#create the minibatch reader and mapping to the features and labels that will be mapped to file content

areader = create_reader(inputfile,True,input_dim,output_dim)

#map input containers (tensors) with the text file structure
reader_map = {
    input_var : areader.streams.features,
    label_var : areader.streams.labels
}



In [10]:
#let define a network and learner  - note how inputs are not included directly in the initial definition 'amodel'
#mbs, minibatch schedule, is ignored for this one as the data is so small, may be useful in other situations
amodel     = Sequential ([Dense(hidden2_dim,activation=C.sigmoid),Dense(output_dim)])
zz         = amodel(input_var) 
mbsize     = 1      #original brainscript was 1
numbatches = 18000  #same as the brainscript (500)*25 per batch
crossent   = C.cross_entropy_with_softmax(zz,label_var)
classerror = C.classification_error(zz,label_var)
learn_rate = learning_rate_schedule(0.04,UnitType.minibatch) #.04 used in brainscript
prprint    = ProgressPrinter(0)                              #0 means a geometric print schedule
mbs        = minibatch_size_schedule([1,2],3000)             #use 1 for the first 1000 samples and then 2 after that

#print("Model: ",amodel, " Inputs: ",amodel.inputs) #just to see if the network looks like our 2 in, 5 hidden etc
Trainer2   = C.Trainer(zz,(crossent,classerror), [sgd(zz.parameters,lr=learn_rate)],[prprint])

agg_loss   = 0.0    #need to add this, testing, and prediction

for ii in range(numbatches):
    mbatch = areader.next_minibatch(mbsize,input_map=reader_map)
    Trainer2.train_minibatch(mbatch)
print('training done')


 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.04
    0.699      0.699          0          0             1
    0.929       1.04      0.333        0.5             3
     1.07       1.18      0.429        0.5             7
     1.03          1        0.4      0.375            15
    0.967      0.905      0.355      0.312            31
     0.93      0.894      0.349      0.344            63
    0.945      0.959      0.386      0.422           127
    0.892       0.84      0.373      0.359           255
    0.814      0.736      0.364      0.355           511
    0.711      0.607      0.305      0.246          1023
      0.6       0.49      0.279      0.253          2047
    0.489      0.378       0.24      0.202          4095
    0.385       0.28      0.172      0.104          8191
    0.301      0.216      0.128     0.0835         16383


with training done we can know try our test data file - same structure as training data just fresh content
testdata.txt contains the test data

In [11]:
#load and process test data - use different approach since CNTK offers so many ways
testinput =  "c:\\Users\\jiwillia\\Documents\\meetup\\deepL\\CNTK-session2\\testdata.txt"
#sample entry from file:    |features 1.0 1.0 |labels 1 0 0

testreader = create_reader(testinput,False,input_dim,output_dim)

test_mb_size = 1 
sample_size  = 9
mb_to_test   = sample_size
test_results = 0.0 
for tt in range(sample_size):
    mbtest = testreader.next_minibatch(test_mb_size,input_map=reader_map)
    #print(mbtest)
    eval_error = Trainer2.test_minibatch(mbtest)
    test_results += eval_error
    #print("EE : ",eval_error)
    
print((test_results/sample_size) * 100, " error percent for ",sample_size, "test samples")


11.11111111111111  error percent for  9 test samples


In [12]:
#let's try to predict skipping the testing phase for now
unknown = np.array([[[9.0,1.0]]],dtype=np.float32) #equivalent to features 4.0, 7.0 labels -1,-1,-1 if this was in a file

prediction = zz.eval({input_var : unknown})

print("predicted array : ",prediction)
print("softmax of predicted is: ",np.argmax(prediction)," :as % ",C.softmax(prediction).eval())
print('done...')

predicted array :  [[ 0.22659075 -4.40430784  4.7554059 ]]
softmax of predicted is:  2  :as %  [[  1.06770927e-02   1.04058927e-04   9.89218891e-01]]
done...
